# Modeling the problem
Since we assume, that we won't know the weather beforehand, the problem could be modeled as the Hidden Markov Model in the following way: <br>
we have a time series of the weather attributes, $X_t$. These influence the number of calls: $X_t \to Y_t$.<br>
$\quad Y_t \quad Y_{t+1}$ <br>
$\quad \hat{|}\qquad \hat{|}$ <br>
$\to X_t \to X_{t+1} \to ...$ <br>
$Y_t$ is also influenced by the some other factors $\tilde X_{t}$, related to human activity, e.g. weekend or season, which are known beforehand for each timestamp. <br>
In the end, our problem consist of 2 problems:<br>
a) predict $X_{t+1}$ based on $X_t, X_{t-1},....$<br>
b) find $f$ s.t. $f(X_{t+1},\tilde X_{t+1})=Y_{t+1}$<br>

As each attribute of the weather is a separate time series, in order to avoid extra work, we will start with b) to understand which time series $X_t$ we don't need to analyze, as they don't contribute much to the model in a)
# Our plan

### 1) derive/predict the number of calls given the weather, time range

For this exactly we will use the **Gradient Boosting** method. Using it we will see which weather attributes contribute most to the model and work with them in the step 2: <br>

###  2) Every weather attribute (selected from 1) is a separate time series, for which we will need the prediction. 
(But we will come to this later)
 Here we will use **ARMA or ARIMA** models, depending on the time series.

----------------

# Step 1: Exploring & preprocessing raw data
Please find the notebook **'Step1_process_raw_data.ipynb'**

# Step 2: Feature engineering
### Step 2.1 Calls
Please find the notebook  **'Step2_calls_data.ipynb'**
### Step 2.2 Weather
Please find the notebook  **'Step2_weather_data.ipynb'**

# Step 3 Model: Predict #calls given weather and date

## Predicting number of calls based on the weather and other attributes

What data we need: <br>
Indep. variables <br>
-- weather attributes<br>
-- extra categorical attributes, derived from the timestamp, that could reflect human activity (see the analysis of time series of number of calls):<br>
* hour of the day 
* day of the week  
* month of the year 
Note that we include it not because of the weather, but because of the human activity at different time of year, e.g. in summer people do more campfires than in winter<br>
* season


Depend.var.<br>
-- number of calls at time t<br>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pyexasol
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels as sm

In [ ]:
def add_dummies(df,attribute):
    # to have categorical values, we need to have the datatype string
    #making sure the datatype is string
    conv_to_str = lambda x: str(x)
    df[attribute]=df[attribute].apply(conv_to_str)
    dummies = pd.get_dummies(df[[attribute]], drop_first=True) #  #dummy var. = #categorical values - 1
    df = df.drop(attribute,axis=1)
    df = pd.concat([df, dummies], axis=1)
    return df

In [ ]:
ds_raw = pd.read_csv('files/weather_to_numb_calls.csv',sep=';',decimal=',')
ds_raw= ds_raw.drop_duplicates(subset=['DT_ISO'])
ds_raw = ds_raw.reset_index(drop=True)
ds_raw['DT_ISO'] = pd.to_datetime(ds_raw.DT_ISO)
# as discussed before we exclude it, as it's highly correlated
columns_to_drop = ['FEELS_LIKE','TEMP_MIN','TEMP_MAX']
ds_raw = ds_raw.drop(columns_to_drop,axis = 1) 
ds_raw = ds_raw.rename(columns={"WEATHER_DESCRIPTION": "wd"}) # shorten the name beforehand
categorical_attr=['wd','H','D','M','S']
for attr in categorical_attr:
    ds_raw=add_dummies(ds_raw,attr)
# some column names have extra spaces in their name
for col in ds_raw.columns:
    ds_raw = ds_raw.rename(columns={col: col.strip()})

# Train & evaluate the model
#### Rmk
To be fair, we won't use the data, that will be our test set for the problem of time series prediction. Thus we should exclude the last year: from 2019-11-01 till 2020-11-01. 

In [ ]:
ds = ds_raw[(ds_raw['DT_ISO']<'2019-11-01')]

In [ ]:
X = ds.drop(['DT_ISO','NUMB_CALLS'], axis = 1) #'BID',,'BIDDING_MODE'
y = ds['NUMB_CALLS']

Now we will apply **gradient boosting for regression** and we will use the scikit-learn library. We will use the **cross validation** to evaluate our model, where the evaluation metric we chose are: **mean squared error and mean absolute error**.

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor  #GBM algorithm
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import cross_val_score, RepeatedKFold #Perforing grid search

In [ ]:
def fit_evaluate(X, y,use_params=False):
    reg = GradientBoostingRegressor()
    reg.fit(X, y)
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores_mse = cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
    n_scores_mae = cross_val_score(reg, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    print('MSE: mean %.3f, st.dev %.3f' % (np.mean(n_scores_mse), np.std(n_scores_mse)))
    print('MAE: mean %.3f, st.dev %.3f' % (np.mean(n_scores_mae), np.std(n_scores_mae)))
    return reg

At first let us see the result with the parameters chosen by default in scikit learn for Gradient Boosting:

In [ ]:
model_gb = fit_evaluate(X, y)

Let us look at the histograms of calls as a baseline:

In [ ]:
plt.hist(y, density=True, bins=50)
plt.show()

Calls are normally distributed with mean around 12 and range roughly between 0 and 30. <br>
Having the absolute error mean 3 that our model is normally off by 3 calls per hour in average. A pretty small standart deviation means this error doesn't deviate much.

Let us find out the **most important features**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [ ]:
from sklearn.inspection import permutation_importance
feature_importance = model_gb.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(20, 16))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(X_train.columns)[sorted_idx])
plt.title('Feature Importance (MDI)')

result = permutation_importance(model_gb, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(X_train.columns)[sorted_idx])
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

The impurity-based and permutation methods show that identifying the **hour of the day and TEMP** are the most significant attributes. As expected we also see **D_Saturday,D_Sunday and S_summer** (which makes sense, as people go out then more). Both graphs show that the **humidity** are also important, followed by **wind_speed, wind_deg and clouds_all**

Let's see if leaving only the most important features would affect significantly the performance:

In [ ]:
selected_features = [col for col in X.columns if col.startswith('H_') ]
other_features=['D_sunday','D_saturday','TEMP','S_summer']
selected_features.extend(other_features)
model_gb_simpler = fit_evaluate(X[selected_features], y)

Previously with all the features we had an error: <br>
MSE: mean -15.100, st.dev 0.346 <br>
MAE: mean -3.025, st.dev 0.036 <br>
As we can see, our model's error barely changed after excluding many parameters.

It means, that now among all the weather parameters, *we will need to predict only* **temperature**. If we found out, that more weather attributes contributed significantly, we would analyze each attributes as a separate time series and have a separate prediction model for each.

We could also apply the grid search to find optimal hyperparameters such as learning rate, maximum number of trees, etc.

# STEP 4: Predict $X_{t+1}$ (temperature) from $X_t$
## Predicting temperature 

We will identify the properties of the temperature time series, decide which lagged information could be used and other additional useful attributes. Once again, we will use the data till November 2019.

In [ ]:
def autocorrelation_attr(df,n_lags=50):
    #plt.rcParams['figure.max_open_warning'] = 0
    weather_time_series = [col for col in df.columns if col not in ('NUMB_CALLS','DT_ISO') ]
    for col in weather_time_series:
        plot_acf(df[col],lags=n_lags)
        plot_pacf(df[col], lags=n_lags)
        plt.title(col)
    plt.show()

In [ ]:
#The **Dickey-Fuller Test** whether this timeseries is **stationary**, 
# i.e. there is no trend or seasonal behaviour (though the series could be cyclic).
def check_stationarity(df):
    print(" > Is the data stationary ?")
    dftest = adfuller(df, autolag='AIC')
    print("Test statistic = {:.3f}".format(dftest[0]))
    print("P-value = {:.3f}".format(dftest[1]))
    print("Critical values :")
    for k, v in dftest[4].items():
        print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<dftest[0] else "", 100-int(k[:-1])))
    return

In [ ]:
#ds contains data till November 2019
temp = ds[['DT_ISO','TEMP']]

In [ ]:
temp.tail()

In [ ]:
temp.hist()

In [ ]:
check_stationarity(temp['TEMP'])

Though the test showed that our data is stationary, this test assumes that there could be only linear trend in the data and thus could be mistaken. 

In [ ]:
def plot_time_series(df,one_col='', omit=[],normalize=False,span=0,calls=False):
    if span>0:
        df = df.set_index("DT_ISO", inplace=False) # inplace=F: make a copy, don't modify directly
        df = df.rolling(span).mean()
        x_axis = df.index
    else:
        x_axis = df['DT_ISO'][:]
    omit.extend(['DT_ISO'])
    columns_time_series=[one_col]
    if len(one_col)==0:
        columns_time_series = [col for col in df.columns if col not in omit ]
    if calls:
        columns_time_series.extend(['NUMB_CALLS'])
    plt.figure("figure",figsize=(40,15))
    for col in columns_time_series:
        if normalize:
            y_axis = (10*df[col]/df[col].max())[:]
            # normalize the output, so all the attributes are between 0 and 10 
        else:
            y_axis = df[col][:]
        plt.plot(x_axis,y_axis)
    plt.legend(columns_time_series)
    plt.grid(True)
    plt.show()
    return

In [ ]:
plot_time_series(temp,one_col='TEMP', omit=[],normalize=False,span=0,calls=False) 

Each year separately

In [ ]:
year_2016=temp[(temp['DT_ISO']<'2017-01-01')&(temp['DT_ISO']>='2016-01-01')]
year_2017=temp[(temp['DT_ISO']>='2017-01-01')&(temp['DT_ISO']<'2018-01-01')]
year_2018=temp[(temp['DT_ISO']>='2018-01-01')&(temp['DT_ISO']<'2019-01-01')]
plot_time_series(year_2016,one_col='TEMP',normalize=False,span=15)
plot_time_series(year_2017,one_col='TEMP',normalize=False,span=15)
plot_time_series(year_2018,one_col='TEMP',normalize=False,span=15)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposed = seasonal_decompose(temp.TEMP, model='additive',extrapolate_trend='freq',freq=365*24)
plt.rcParams.update({'figure.figsize':(10,10)})
decomposed.plot()
plt.show()

Obviously the time series has **yearly seasonality** and possibly a declining trend, thus is not stationary.

In [ ]:
# let's see the autocorrelation for one week
autocorrelation_attr(temp,n_lags=50)

Here we see that the temperature at hour t very strongly depends on the **previous hour/hours** and the tempreture **24 hours before (daily seasonality)**. This is also backed by the partial autocorrelation function. <br>

## Training and testing

As we have the seasonality, unless we remove it, we cannot use ARIMA model for this problem. <br>
However, in that case best model to represent this time series is **seasonal ARIMA** or SARIMA. <br>

Except for the ARIMA's parameters:
* p: Trend autoregression order.
* d: Trend difference order.
* q: Trend moving average order 

we also need to define the seasonal elements:

* P: Seasonal autoregressive order.
* D: Seasonal difference order.
* Q: Seasonal moving average order.
* m: The number of time steps for a single seasonal period. 

To sum up, we have a model, where $X_t$ depends on $X_{t-1}, X_{t-24}$ and $X_{t-365-6}$ (Year has 365+6 hours) <br>
In our case we have daily and yearly seasonality, thus **m = 24** <br>
We added 6, as each year has 365 days and 6 hours and the temperature is not "aware" about the leap year.<br>
In that case P = how many days before we will take into consideration. E.g. P=2 => (t-1 days),(t-2 days) <br>
We will find out what parameters fit best for our model using the **grid search**


**Rmk**: Seasonal ARIMA is designed for shorter periods such as 12 for monthly data or 24 for hourly data. In order to capture the yearly seasonality in hourly data, m has to be tremendously large 24 x 365+6, which is very a computationally heavy (requires too much RAM). This way we expect that our model might not capture the yearly seasonality, but will predict better for short term timerange.<br>

## Model based on one year of training

We will train our model on **one year** of data.

In [ ]:
training_year=temp[(temp['DT_ISO']>='2018-11-01')&(temp['DT_ISO']<'2019-11-01')]

In [ ]:
import warnings
import itertools

In [ ]:
# in case pmdarima is not installed
try:
    import pmdarima as pm
except:
    !pip install pmdarima

In [ ]:
def grid_search_SARMAX(data):
    p = range(0, 3)
    d = range(1,2)
    q = range(0, 3)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 24) for x in list(itertools.product(p, d, q))]
    print('Examples of parameter combinations for Seasonal ARIMA...')
    print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
    print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
    print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
    print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            mod = sm.tsa.statespace.sarimax.SARIMAX(data['TEMP'],
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
    return results

In [ ]:
### 
###  DUE to the limitation of RAM, this was run on Google Colab (see the output in the next cell)
### 
results = grid_search_SARMAX(training_year)

The model **SARIMA(1,1,1) (1,1,1,24)** has the lowest AIC thus that is our choice.

In [ ]:
# Fit the SARIMAX model using optimal parameters
mod = sm.tsa.statespace.sarimax.SARIMAX(training_year['TEMP'],
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 24),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

Though the residuals seem to be normally distributed, the correlogram shows that some residuals are correlated with some lags. As we said before Seasonal ARIMA is designed to learn short periods of seasonality. We could tackle it by applying the Fourier series approach.  <br> 
(However we will stop here due to time and memory limitations). <br> 

In [ ]:
# Fit the SARIMAX model with the correct time index
training_year_index = training_year.set_index("DT_ISO", inplace=False)
training_year_index.index = pd.DatetimeIndex(training_year_index.index).to_period('H')

In [ ]:
model = sm.tsa.statespace.sarimax.SARIMAX(training_year_index,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 24),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = model.fit()

## Backtesting
One of the form of backtesting is **walk forward validation**. We will make a prediction for one hour ehead and then use this prediction for predicting further.

In [ ]:
import datetime as dt
# in start and end we need datetime.datetime 
prediction = results.get_prediction(start= dt.datetime(2019, 11, 1,0,0), 
                            end= dt.datetime(2020, 11, 1,0,0),
                            dynamic=False) 
# The dynamic=False argument ensures that we produce one-step ahead forecasts, 
# meaning that forecasts at each point are generated using the full history up to that point.
prediction_ci = prediction.conf_int()

In [ ]:
# now let's get the training & test data
testing_year = ds_raw[(ds_raw['DT_ISO']>='2019-11-01')][['DT_ISO','TEMP']]
testing_year_index = testing_year.set_index("DT_ISO", inplace=False)
testing_year_index.index = pd.DatetimeIndex(testing_year_index.index).to_period('H')
# combine the test year (2020) and the training years (2018,2019) 
all_series = pd.concat([training_year_index, testing_year_index], axis=0)

In [ ]:
# in case some duplicate indices appeared
all_series = all_series[~all_series.index.duplicated(keep='first')]

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10)})
ax = all_series['2018-11-01':].plot(label='observed')
prediction.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)
ax.fill_between(prediction_ci.index,
                    prediction_ci.iloc[:, 0],
                    prediction_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Temp')
plt.legend()
plt.show()

The confidence interval is growing very rapidly and the prediction becomes very uncertain even at the early stage.Let us take a closer look at the prediction for one month: 

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10)})
ax = all_series['2018-11-01':].plot(label='observed')
prediction.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)
ax.set_xlabel('Date')
ax.set_ylabel('Temp')
plt.legend()
plt.show()

As we can see, that model cannot capture the long-term seasonality, as it is based on the short-term ones. As we stated above, we did not train SARIMA with very high the parameter m=365x24 that would capture yearly seasonality due to memory limitations. <br> <br>
It is a known fact, that ARIMA models converge either to straight line, non-zero constant or to mean of the data, depending on the parameters of the model. That is what we see in our case: at first the model makes more reasonable predictions and then just follows a straight line. <br>

Let us look closer at the prediction in November

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10)})
prediction2 = results.get_prediction(start= dt.datetime(2019, 11, 1,0,0), 
                            end= dt.datetime(2019, 12, 1,0,0),
                            dynamic=False)
ax = all_series['2019-10-01':'2019-12-01'].plot(label='observed')
prediction2.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)
ax.set_xlabel('Date')
ax.set_ylabel('Temp')
plt.legend()
plt.show()

The prediction mimics the behaviour from around 1 week ago, where we observe the declining trend in real data. 

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
# Compute the mean square error
y_forecasted = prediction2.predicted_mean[:-1] # the last element is redundant
y_truth = all_series['2019-11-01':'2019-11-30']
mse = np.sqrt(MSE(y_truth, y_forecasted).mean())
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

**Prediction for a week**

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10)})
prediction3 = results.get_prediction(start= dt.datetime(2019, 11, 1,0,0), 
                            end= dt.datetime(2019, 11, 8,0,0),
                            dynamic=False)
ax = all_series['2019-10-20':'2019-11-08'].plot(label='observed')
prediction_ci_3 = prediction3.conf_int()
prediction3.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)
ax.fill_between(prediction_ci_3.index,
                    prediction_ci_3.iloc[:, 0],
                    prediction_ci_3.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Temp')
ax.set_xlabel('Date')
ax.set_ylabel('Temp')
plt.legend()
plt.show()

In [ ]:
# Compute the mean square error
y_forecasted = prediction3.predicted_mean[:-1] # the last element is redundant
y_truth = all_series['2019-11-01':'2019-11-07']
mse = np.sqrt(MSE(y_truth, y_forecasted).mean())
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

## Model based on one month of training

In [ ]:
training_month = training_year[training_year['DT_ISO']>'2019-10-01']

In [ ]:
training_month

In [ ]:
grid_search_SARMAX(training_month)

We will take the model  **ARIMA(1, 1, 2)x(1, 1, 2, 24)** with AIC score 970.

In [ ]:
# Fit the SARIMAX model with the correct time index
model = sm.tsa.statespace.sarimax.SARIMAX(training_month['TEMP'],
                                order=(1, 1, 2),
                                seasonal_order=(1, 1, 2, 24),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results_month = model.fit()

In [ ]:
results_month.plot_diagnostics(figsize=(15, 12))
plt.show()

Now this model performance looks better than the previous one: here we see that the residuals are normally distributed with the mean zero and almost not autocorrelated.

In [ ]:
training_month_index = training_year_index['2019-10-01':'2019-10-31']
training_month_index = training_month_index[~training_month_index.index.duplicated(keep='first')]

In [ ]:
# Fit the SARIMAX model with the correct time index
model = sm.tsa.statespace.sarimax.SARIMAX(training_month_index,
                                order=(1, 1, 2),
                                seasonal_order=(1, 1, 2, 24),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results_month = model.fit()

In [ ]:
# in start and end we need datetime.datetime 
prediction_month = results_month.get_prediction(start= dt.datetime(2019, 11, 1,0,0), 
                            end= dt.datetime(2019, 12, 1,0,0),
                            dynamic=False) 
# The dynamic=False argument ensures that we produce one-step ahead forecasts, 
# meaning that forecasts at each point are generated using the full history up to that point.
prediction_ci_month = results_month.conf_int()

In [ ]:
prediction_month.predicted_mean

In [ ]:
all_series = all_series[~all_series.index.duplicated(keep='first')]

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10)})
ax = all_series['2019-10-01':'2019-12-01'].plot(label='observed')
prediction_month.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)
ax.set_xlabel('Date')
ax.set_ylabel('Temp')
plt.legend()
plt.show()

The model predicts well only for some next days, but captures a decreasing trend. <br> 
We can see the the error is smaller in comparison to the model trained on more data. (see above) <br>
As an idea, to improve further the result we could test the Fourier series approach or even using Gradient Boosting with variables, denoting the seasonal patterns (in our case yearly and daily patterns of temperature change), but for now we will keep the last SARIMA model.

In [ ]:
# Compute the mean square error
y_forecasted = prediction_month.predicted_mean[:-1] # the last element is redundant
y_truth = all_series['2019-11-01':'2019-11-30']
mse = np.sqrt(MSE(y_truth, y_forecasted).mean())
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

**Predict for a week**:

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10)})
prediction_week = results_month.get_prediction(start= dt.datetime(2019, 11, 1,0,0), 
                            end= dt.datetime(2019, 11, 8,0,0),
                            dynamic=False)
ax = all_series['2019-10-27':'2019-11-07'].plot(label='observed')
prediction_week_ci = prediction_week.conf_int()
prediction_week.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)
ax.fill_between(prediction_week_ci.index,
                    prediction_week_ci.iloc[:, 0],
                    prediction_week_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('Temp')
plt.legend()
plt.show()

In [ ]:
type(prediction_week.predicted_mean)

In [ ]:
# Compute the mean square error
y_forecasted = prediction_week.predicted_mean[:-1] # the last element is redundant
y_truth = all_series['2019-11-01':'2019-11-07']
mse = np.sqrt(MSE(y_truth, y_forecasted).mean())
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

**The predictions for a week and a month are at about as precise for both models: the one trained with one month of data and the one trained with one year of data.**

# Last step: Evaluate both models working together

As we are going to use the **model trained on one month** and the simpler model of Gradient boosting. <br>

The Gradient boosting model will take as an input <br>

date | temperature | hour | is Sunday | is Saturday | is summer

In [ ]:
# we will use our predictor for temperature trained on one month
# results_month

# and the above trained Gradient boosting model 
# model_gb_simpler 

In [ ]:
from datetime import datetime

In [ ]:
def validate_time_input(time_str):
    try:
        time_from = datetime.strptime(time_from_str, "%Y-%m-%d %H")
    except:
        print("Please input the correct format: YYYY-MM-DD HH")
        print("E.g. 2019-11-01 01")
        return False
    return True      

In [ ]:
def predict_temperature(time_from_str, time_till_str): # 'YYYY-MM-DD HH'
    if validate_time_input(time_from_str) and validate_time_input(time_till_str):
        time_from = datetime.strptime(time_from_str, "%Y-%m-%d %H")
        time_till = datetime.strptime(time_till_str, "%Y-%m-%d %H")
    else:
        return
    start = dt.datetime(time_from.year,time_from.month,time_from.day,time_from.hour,0)
    end = dt.datetime(time_till.year,time_till.month,time_till.day,time_till.hour,0)
    prediction = results_month.get_prediction(start= start,end= end,dynamic=False)
    return prediction.predicted_mean

In [ ]:
selected_features = ['H_1', 'H_10', 'H_11', 'H_12', 'H_13', 'H_14', 'H_15', 'H_16', 'H_17', 'H_18', \
                     'H_19', 'H_2', 'H_20', 'H_21', 'H_22', 'H_23', 'H_3', 'H_4', 'H_5', 'H_6', 'H_7', \
                     'H_8', 'H_9', 'D_sunday', 'D_saturday', 'TEMP', 'S_summer']

def define_input(time_from_str, time_till_str):
    if validate_time_input(time_from_str) and validate_time_input(time_till_str):
        time_from = datetime.strptime(time_from_str, "%Y-%m-%d %H")
        time_till = datetime.strptime(time_till_str, "%Y-%m-%d %H")
    else:
        return
    time_range = pd.Series(pd.date_range(start = time_from_str,end=time_till_str, freq='H'))
    input_df = pd.DataFrame(0, index=np.arange(len(time_range)), columns=selected_features)
    input_df['DT_ISO']=time_range
    hours = input_df.DT_ISO.dt.hour
    months = input_df.DT_ISO.dt.month
    days_of_week=input_df.DT_ISO.dt.dayofweek
    i=0
    for i in range(len(input_df)):
        if hours[i]>0: # hour=0 corresponds to H_1=0...H_23=0
            col_h='H_'+str(hours[i])
            input_df.loc[i,col_h]=1
        if months[i]>=6 and months[i]<=8:
            input_df.loc[i,'S_summer']=1
        if days_of_week[i]==6:
            input_df.loc[i,'D_sunday']=1
        elif days_of_week[i]==5:
            input_df.loc[i,'D_saturday']=1
    predicted_temp = predict_temperature(time_from_str, time_till_str)
    if len(predicted_temp)==len(input_df['TEMP']):
        for i in range(len(X_validate['TEMP'])):
            input_df.loc[i,'TEMP'] = predicted_temp[i]
    input_df = input_df.drop(['DT_ISO'],axis=1)
    return input_df     

In [ ]:
def get_validation_vector(df_real,time_from_str, time_till_str):
    if validate_time_input(time_from_str) and validate_time_input(time_till_str):
        df_validate = df_real[(df_real['DT_ISO']>=time_from_str) & (df_real['DT_ISO']<=time_till_str)]
        return df_validate['NUMB_CALLS']

In [ ]:
def test_model(df_real,time_from_str, time_till_str,plot=False):
    if validate_time_input(time_from_str) and validate_time_input(time_till_str):
        input_df = define_input(time_from_str,time_till_str)
        validation_vector = get_validation_vector(df_real,time_from_str,time_till_str)
        call_prediction = model_gb_simpler.predict(input_df)
        mse = np.sqrt(MSE(validation_vector, call_prediction).mean())
        return call_prediction, mse
    else:
        return

In [ ]:
time_from_str = '2019-11-01 01'
time_till_str ='2020-10-31 23'
call_prediction, mse = test_model(ds_raw,time_from_str, time_till_str,plot=True)
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))